## 허깅 페이스 모델을 양자화 하고 Lora 로 한국어 학습 시키는 샘플

In [1]:
!pip install -q ipywidgets

In [11]:
!pip install -q transformers
!pip install -q mlx_lm
!pip install -q jinja2
!pip install -q datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

### Tiny Llama 1.1B

#### 모델 양자화

In [3]:
!python ./mlx-examples/lora/convert.py --hf-path TinyLlama/TinyLlama-1.1B-Chat-v1.0 --mlx-path ./my_models/mlx_tinyllama -q

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
[INFO] Loading
Fetching 8 files: 100%|███████████████████████████| 8/8 [00:00<00:00, 15.94it/s]
[INFO] Quantizing


#### 모델 동작 확인

In [5]:
from mlx_lm import load, generate

model, tokenizer = load("./my_models/mlx_tinyllama")

In [6]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
response = generate(model, tokenizer, prompt=prompt, verbose=True)

Prompt: <|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>

There is no definitive answer as to how many helicopters a human can eat in one sitting. However, it is generally accepted that humans are not designed to consume large quantities of food at once. A human's digestive system is designed to process food in small, digestible portions, which can take several hours to fully digest. Therefore, humans typically consume food in small, frequent meals throughout the day. In general, humans can consume up to a few
Prompt: 176.528 tokens-per-sec
Generation: 61.076 tokens-per-sec


In [10]:
messages = [
    {
        "role": "system",
        "content": "당신은 친절한 챗봇입니다. 항상 한국어로만 답을 하세요.",
    },
    {"role": "user", "content": "가장 아름다운 꽃은 무엇인가요?"},
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
response = generate(model, tokenizer, prompt=prompt, verbose=True)

Prompt: <|system|>
당신은 친절한 챗봇입니다. 항상 한국어로만 답을 하세요.</s>
<|user|>
가장 아름다운 꽃은 무엇인가요?</s>
<|assistant|>

가장 아름다운 꽃은 무었습니다. 왜 그런지 알려주세요.
Prompt: 495.670 tokens-per-sec
Generation: 60.501 tokens-per-sec


#### 데이터셋 준비

In [12]:
from datasets import load_dataset

dataset = load_dataset('Bingsu/ko_alpaca_data')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Generating train split:   0%|          | 0/49620 [00:00<?, ? examples/s]

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 49620
    })
})

In [15]:
def convert_to_instruction_output(example):
    instruction = example["instruction"]
    input_text = example["input"]
    output_text = example["output"]
    
    if len(input_text) > 0:
        return {'text': f"<s><INST>{instruction}\n\n{input_text}</INST>{output_text}</s>"}
    else:
        return {'text': f"<s><INST>{instruction}</INST>{output_text}</s>"}

# 변환 함수 적용
converted_dataset = dataset['train'].map(convert_to_instruction_output)

Map:   0%|          | 0/49620 [00:00<?, ? examples/s]

In [19]:
converted_dataset[3:7]

{'instruction': ['대기 오염을 어떻게 줄일 수 있나요?',
  '어려운 결정을 내려야 했던 때를 설명하세요.',
  '홀수 중 하나를 밝히세요.',
  '다음 분수가 1/4에 해당하는 이유를 설명하세요.'],
 'input': ['', '', '트위터, 인스타그램, 텔레그램', '4/16'],
 'output': ['대기 오염을 줄이기 위해 자동차 대신 대중교통을 이용하고, 미세먼지 배출량을 줄이기 위해 차량 정기 검사를 받아야 합니다. 또한 에너지 절약을 위해 전기를 아끼고, 나무를 심어 대기 오염을 줄일 수 있습니다.',
  '제가 어려운 결정을 내려야 했던 때는 대학원 졸업 후 직장을 찾아야 했던 때입니다. 다양한 선택지가 있었지만 나의 관심사와 비전에 가장 부합하는 경력을 뽑는데 어려움이 있었습니다. 이 때 여러 사람들의 조언과 현재 상황에 대한 깊은 고민 끝에 적절한 선택을 내리게 되었습니다.',
  '텔레그램입니다.',
  '분모와 분자를 모두 4로 나눌 수 있기 때문에 1/4에 해당합니다.'],
 'text': ['<s><INST>대기 오염을 어떻게 줄일 수 있나요?</INST>대기 오염을 줄이기 위해 자동차 대신 대중교통을 이용하고, 미세먼지 배출량을 줄이기 위해 차량 정기 검사를 받아야 합니다. 또한 에너지 절약을 위해 전기를 아끼고, 나무를 심어 대기 오염을 줄일 수 있습니다.</s>',
  '<s><INST>어려운 결정을 내려야 했던 때를 설명하세요.</INST>제가 어려운 결정을 내려야 했던 때는 대학원 졸업 후 직장을 찾아야 했던 때입니다. 다양한 선택지가 있었지만 나의 관심사와 비전에 가장 부합하는 경력을 뽑는데 어려움이 있었습니다. 이 때 여러 사람들의 조언과 현재 상황에 대한 깊은 고민 끝에 적절한 선택을 내리게 되었습니다.</s>',
  '<s><INST>홀수 중 하나를 밝히세요.\n\n트위터, 인스타그램, 텔레그램</INST>텔레그램입니다.</s>',
  '<s><INST>다음 분수가 1/4에 해당하는 이유를 설명

In [20]:
keep_features = ['text']
remove_features = [feat for feat in dataset['train'].features if feat not in keep_features]
pruned_dataset = converted_dataset.remove_columns(column_names=remove_features)

In [21]:
pruned_dataset[3:7]

{'text': ['<s><INST>대기 오염을 어떻게 줄일 수 있나요?</INST>대기 오염을 줄이기 위해 자동차 대신 대중교통을 이용하고, 미세먼지 배출량을 줄이기 위해 차량 정기 검사를 받아야 합니다. 또한 에너지 절약을 위해 전기를 아끼고, 나무를 심어 대기 오염을 줄일 수 있습니다.</s>',
  '<s><INST>어려운 결정을 내려야 했던 때를 설명하세요.</INST>제가 어려운 결정을 내려야 했던 때는 대학원 졸업 후 직장을 찾아야 했던 때입니다. 다양한 선택지가 있었지만 나의 관심사와 비전에 가장 부합하는 경력을 뽑는데 어려움이 있었습니다. 이 때 여러 사람들의 조언과 현재 상황에 대한 깊은 고민 끝에 적절한 선택을 내리게 되었습니다.</s>',
  '<s><INST>홀수 중 하나를 밝히세요.\n\n트위터, 인스타그램, 텔레그램</INST>텔레그램입니다.</s>',
  '<s><INST>다음 분수가 1/4에 해당하는 이유를 설명하세요.\n\n4/16</INST>분모와 분자를 모두 4로 나눌 수 있기 때문에 1/4에 해당합니다.</s>']}

In [22]:
import numpy as np

def split_data(dataset, valid_ratio=0.2, seed=42):
    np.random.seed(seed)
    indices = np.arange(len(dataset))
    np.random.shuffle(indices)
    
    valid_indices = indices[:int(len(dataset) * valid_ratio)]
    train_indices = indices[int(len(dataset) * valid_ratio):]
    
    return dataset.select(train_indices), dataset.select(valid_indices)

In [23]:
train_data, valid_data = split_data(pruned_dataset)

In [24]:
train_data

Dataset({
    features: ['text'],
    num_rows: 39696
})

In [25]:
valid_data

Dataset({
    features: ['text'],
    num_rows: 9924
})

In [26]:
output_path = 'my_datasets/Bingsu/ko_alpaca_data/train.jsonl'

train_data.to_json(output_path, orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/40 [00:00<?, ?ba/s]

21117930

In [27]:
output_path = 'my_datasets/Bingsu/ko_alpaca_data/valid.jsonl'

valid_data.to_json(output_path, orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

5229310

#### 모델 파인튜닝

In [29]:
# iteration 수와 layer 수를 적당히 작게 하면 (허접하지만) 빨리 튜닝 결과를 얻어볼 수 있습니다.
# ! python lora/lora.py --model ./my_models/mlx_ywko_tinyllama --train --iters 10 --data ./my_datasets/Bingsu/ko_alpaca_data --lora-layer 1
#
# 그리고 100 iteration 마다 중간 결과를 adapters.npz 에 저장하므로 별도로 노트북을 열어서 다음 셀 이후의 작업을 실행시키면 중간 학습 과정에서의 결과물을 미리 보실 수 있습니다. 
#
! python ./mlx-examples/lora/lora.py --model ./my_models/mlx_tinyllama --train --data ./my_datasets/Bingsu/ko_alpaca_data 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Loading pretrained model
Total parameters 228.076M
Trainable parameters 0.819M
Loading datasets
Training
Iter 1: Val loss 1.805, Val took 43.673s
Iter 10: Train loss 1.636, It/sec 0.290, Tokens/sec 223.683
Iter 20: Train loss 1.657, It/sec 0.322, Tokens/sec 216.484
Iter 30: Train loss 1.376, It/sec 0.286, Tokens/sec 174.584
Iter 40: Train loss 1.350, It/sec 0.318, Tokens/sec 204.820
Iter 50: Train loss 1.371, It/sec 0.274, Tokens/sec 194.649
Iter 60: Train loss 1.359, It/sec 0.302, Tokens/sec 200.184
Iter 70: Train loss 1.253, It/sec 0.283, Tokens/sec 203.665
Iter 80: Train loss 1.250, It/sec 0.283, Tokens/sec 195.227
Iter 90: Train loss 1.307, It/sec 0.278, Tokens/sec 199.544
Iter 100: Train loss 1.161, It/sec 0.276, Tokens/sec 215.837
Iter 100: Saved adapter weights to adapters.npz.
Iter 110: Train loss 1.234, It/sec 0.327, Tok

In [34]:
!mv adapters.npz ./my_models/mlx_tinyllama_01.npz

mv: adapters.npz: No such file or directory


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [35]:
!python ./mlx-examples/lora/fuse.py --model ./my_models/mlx_tinyllama --save-path ./my_models/mlx_tinyllama_ft_01 --adapter-file ./my_models/mlx_tinyllama_01.npz --hf-path TinyLlama/TinyLlama-1.1B-Chat-v1.0 


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Loading pretrained model


#### 파인튜닝 결과 확인

In [36]:
from mlx_lm import load, generate

model_ft, tokenizer_ft = load("./my_models/mlx_tinyllama_ft_01")


In [37]:
messages = [
    {
        "role": "system",
        "content": "당신은 친절한 챗봇입니다. 항상 한국어로만 답을 하세요.",
    },
    {"role": "user", "content": "가장 아름다운 꽃은 무엇인가요?"},
]

prompt = tokenizer_ft.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
response = generate(model_ft, tokenizer_ft, prompt=prompt, verbose=True)

Prompt: <|system|>
당신은 친절한 챗봇입니다. 항상 한국어로만 답을 하세요.</s>
<|user|>
가장 아름다운 꽃은 무엇인가요?</s>
<|assistant|>

가장 아름다운 꽃은 매우 많은 것들이 있습니다. 예를 들어, 매우 많은 양념의 매우 많은 양념의 매우 많은 
Prompt: 499.790 tokens-per-sec
Generation: 58.987 tokens-per-sec


### Mistral 7B

#### 모델 양자화

In [38]:
!python ./mlx-examples/lora/convert.py --hf-path mistralai/Mistral-7B-Instruct-v0.2 --mlx-path ./my_models/mlx_mistral_7b -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
[INFO] Loading
Fetching 11 files: 100%|█████████████████████| 11/11 [00:00<00:00, 86724.33it/s]
[INFO] Quantizing


#### 모델 동작 확인

In [39]:
from mlx_lm import load, generate

model, tokenizer = load("./my_models/mlx_mistral_7b")


In [40]:
response = generate(model, tokenizer, prompt="가장 아름다운 꽃은 무엇인가요?", verbose=True)

Prompt: 가장 아름다운 꽃은 무엇인가요?
그 맹 맹 맹 맹 맹 맹 맹 맹 맹 맹 맹 맹 맹 맹 맹 맹 맹 맹 맹 맹 맹 맹 맹 맹 
Prompt: 67.458 tokens-per-sec
Generation: 16.508 tokens-per-sec


In [ ]:
response = generate(model, tokenizer, prompt="가장 아름다운 꽃은 무엇인가요?", verbose=True)

Prompt: 가장 아름다운 꽃은 무엇인가요?
(What is the most beautiful flower?) is a common question that has been asked throughout history. The answer, of course, depends on personal preferences and cultural backgrounds. However, there are some flowers that are universally recognized for their beauty and symbolism. Here are ten of the most beautiful flowers in the world:

1. Rose: The rose is often considered the queen of flowers due to its timeless beauty and symbolism. With their delicate petals, sweet fragrance,
Prompt: 0.909 tokens-per-sec
Generation: 0.476 tokens-per-sec


In [ ]:
response = generate(model, tokenizer, prompt="당신은 친절한 챗봇입니다. 항상 한국어로만 답을 하세요. 가장 아름다운 꽃은 무엇인가요?", verbose=True)

Prompt: 당신은 친절한 챗봇입니다. 항상 한국어로만 답을 하세요. 가장 아름다운 꽃은 무엇인가요?


안녕하세요! 가장 아름다운 꽃은 장미입니다. 장미는 멸측되지 않고 지속적으로 피어나는 꽃으로, 멸측되지 않는 아름다움과 지속적인 존재감에서 느
Prompt: 3.235 tokens-per-sec
Generation: 0.941 tokens-per-sec


In [ ]:
response = generate(model, tokenizer, prompt="당신은 친절한 챗봇입니다. 항상 한국어로만 답을 하세요. 건강을 지키는 비결은 무엇인가?", verbose=True)

Prompt: 당신은 친절한 챗봇입니다. 항상 한국어로만 답을 하세요. 건강을 지키는 비결은 무엇인가?


안녕하세요! 건강을 지키는 비결에는 많은 것이 있지만, 일상적인 생활에서 가장 중요한 것들을 말하겠습니다.

1. 건강한 식단 섭취: 과일, 채소, 견고한 단백질
Prompt: 76.045 tokens-per-sec
Generation: 9.859 tokens-per-sec


In [ ]:
response = generate(model, tokenizer, prompt="당신은 친절한 챗봇입니다. 항상 한국어로만 답을 하세요. 주요 프로그래밍 언어는 어떤 것이 있나요?", verbose=True)

Prompt: 당신은 친절한 챗봇입니다. 항상 한국어로만 답을 하세요. 주요 프로그래밍 언어는 어떤 것이 있나요?


대부분의 프로그래밍 언어가 있습니다. C, Java, Python, JavaScript, Ruby, Swift 등이 있습니다. 이 중에서도 한국어로 프로그래밍 언어를 찾으시면 다음과 같습니다.

1. 코틀린 (Kotlin
Prompt: 70.124 tokens-per-sec
Generation: 8.449 tokens-per-sec


#### 모델 파인튜닝

In [41]:
! python ./mlx-examples/lora/lora.py --model ./my_models/mlx_mistral_7b --train --data ./my_datasets/Bingsu/ko_alpaca_data

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Loading pretrained model
Total parameters 1244.041M
Trainable parameters 1.704M
Loading datasets
Training
Iter 1: Val loss 2.639, Val took 191.334s
Iter 10: Train loss 2.122, It/sec 0.071, Tokens/sec 41.529
Iter 20: Train loss 1.788, It/sec 0.082, Tokens/sec 42.068
Iter 30: Train loss 1.549, It/sec 0.077, Tokens/sec 35.038
Iter 40: Train loss 1.502, It/sec 0.080, Tokens/sec 39.123
Iter 50: Train loss 1.459, It/sec 0.068, Tokens/sec 37.114
Iter 60: Train loss 1.489, It/sec 0.073, Tokens/sec 36.748
Iter 70: Train loss 1.402, It/sec 0.076, Tokens/sec 40.875
Iter 80: Train loss 1.398, It/sec 0.076, Tokens/sec 38.947
Iter 90: Train loss 1.388, It/sec 0.076, Tokens/sec 41.874
Iter 100: Train loss 1.272, It/sec 0.069, Tokens/sec 40.874
Iter 100: Saved adapter weights to adapters.npz.
Iter 110: Train loss 1.358, It/sec 0.088, Tokens/sec 

In [42]:
!mv adapters.npz ./my_models/mlx_mistral_01.npz

Python(43895) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [43]:
!python ./mlx-examples/lora/fuse.py --model ./my_models/mlx_mistral_7b --save-path ./my_models/mlx_mistral_7b_ft_01 --adapter-file ./my_models/mlx_mistral_01.npz --hf-path mistralai/Mistral-7B-Instruct-v0.2 


Python(43896) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Loading pretrained model


#### 파인튜닝 결과 확인

In [1]:
from mlx_lm import load, generate

model_ft, tokenizer_ft = load("./my_models/mlx_mistral_7b_ft_01")


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
response = generate(model_ft, tokenizer_ft, prompt="당신은 친절한 챗봇입니다. 항상 한국어로만 답을 하세요. 가장 아름다운 꽃은 무엇인가요?", verbose=True)

Prompt: 당신은 친절한 챗봇입니다. 항상 한국어로만 답을 하세요. 가장 아름다운 꽃은 무엇인가요?

No tokens generated for this prompt


In [3]:
response = generate(model_ft, tokenizer_ft, prompt="가장 아름다운 꽃은 무엇인가요?", verbose=True)

Prompt: 가장 아름다운 꽃은 무엇인가요?


가장 아름다운 꽃은 빛나는 황색 꽃입니다.
Prompt: 66.969 tokens-per-sec
Generation: 17.098 tokens-per-sec


In [4]:
response = generate(model_ft, tokenizer_ft, prompt="건강을 지키는 비결은 무엇인가?", verbose=True)

Prompt: 건강을 지키는 비결은 무엇인가?


건강을 지키는 비결은 건강한 식단, 충분한 수면, 적절한 운동, 스트레스 관리 등이 있습니다.
Prompt: 66.138 tokens-per-sec
Generation: 17.030 tokens-per-sec


In [5]:
response = generate(model_ft, tokenizer_ft, prompt="주요 프로그래밍 언어는 어떤 것이 있나요?", verbose=True)

Prompt: 주요 프로그래밍 언어는 어떤 것이 있나요?




Python, Java, C++, Ruby, JavaScript, Swift, Go, R, PHP, C#

이 프로그래밍 언어들은 모두 다른 방식으로 프로그래밍을 할 수 있습니다.
Prompt: 89.954 tokens-per-sec
Generation: 16.641 tokens-per-sec
